In [1]:
import os
import re
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import json
import math
import numpy as np
from bokeh.io import show, output_file
from bokeh.plotting import figure

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
import pandas as pd

from bokeh.models import HoverTool
from numpy import histogram, linspace

from IPython.display import display, HTML

%matplotlib inline
%config IPCompleter.greedy=True

pd.set_option('display.max_colwidth', -1)

In [2]:
data_dir='/Users/genie/dev/projects/github/network_analysis_ticket_to_ride/data/'
destinations_path = os.path.join(data_dir,'destinations.txt')
routes_path = os.path.join(data_dir,'trainroutes.txt')
output_dir='/Users/genie/dev/projects/github/network_analysis_ticket_to_ride/output/'

In [3]:
pattern = re.compile('\\w+')

In [4]:
Gm = nx.MultiGraph()

In [5]:
# functions

# function to calculate points from segment length
def points(distance):
    x = [0, 1, 2, 4, 7, 0, 15, 0, 21]
    return x[distance]

def inverse_weight(distance):
    return(1/distance)

# distance of path p in graph g
def distance_of_path(g, p):
    d = sum([g.edges[p[e],p[e+1]]['distance'] for e in range(len(p)-1)])
    return d

# points of path p in graph g
def points_of_path(g, p):
    d = sum([g.edges[p[e],p[e+1]]['points'] for e in range(len(p)-1)])
    return d

# return all paths between u and v in graph g up to distance min(u,v)+2
def alternate_scoring_paths_with_cutoff(g, u, v, cutoff):
    sp_length = nx.shortest_path_length(g,u,v)
    paths = nx.all_simple_paths(g, u, v, cutoff=sp_length+2)
    return list(paths)

In [6]:
# construct graph

Gm.clear()

with open(routes_path) as f:
    for line in f:
        city1, city2, distance, route_type, color, is_multi = pattern.findall(line)
        distance = int(distance)
        if(Gm.has_edge(city1,city2)==False):
            Gm.add_edge(city1, city2, key=0, distance=distance, route_type=route_type, color=color, points=points(distance), 
                   weight=distance, importance=0)

G = nx.Graph(Gm)

## Structure

In [7]:
from bokeh.models import Range1d, Plot
from bokeh.models.graphs import from_networkx
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.models.annotations import Title
from bokeh.models import Circle, HoverTool, MultiLine
from bokeh.plotting import figure, show, save
from bokeh.io import output_file, output_notebook
from bokeh.models import Div
from bokeh.layouts import gridplot, column

#output_notebook()
output_file(output_dir + 'node_network.html', mode='inline')

# normalize node size
degree_list = [G.degree[n] for n in G.nodes(data=False)]
start = 10
end = 25
min_ex = np.min(degree_list)
max_ex = np.max(degree_list)
degree_dict = {}
for n,v in sorted(Gm.degree, key=lambda x: x[1], reverse=True):
    norm_v = ((v-min_ex)/(max_ex-min_ex)) * (end-start) + start
    degree_dict[n] = norm_v

# Set node attributes
#nx.set_node_attributes(Gm, 'node_color', node_color)
nx.set_node_attributes(Gm, degree_dict, 'node_size')

# We could use figure here but don't want all the axes and titles
plot = Plot(x_range=Range1d(-2, 2), y_range=Range1d(-2 ,2), title=None) #title=Title(text="Network"))
#plot.title.text="Network"
plot.width = 900;

# Create a Bokeh graph from the NetworkX input using nx.spring_layout
graph = from_networkx(Gm, nx.spring_layout, scale=1.8, center=(0,0))
plot.renderers.append(graph)

# Blue circles for nodes, and light grey lines for edges
graph.node_renderer.glyph = Circle(size='node_size', fill_color='#2b83ba')
graph.edge_renderer.glyph = MultiLine(line_color="#cccccc", line_alpha=0.8, line_width=2)

# green hover for both nodes and edges
graph.node_renderer.hover_glyph = Circle(size=25, fill_color='#abdda4')
graph.edge_renderer.hover_glyph = MultiLine(line_color='#abdda4', line_width=4)

# When we hover over nodes, highlight adjecent edges too
graph.inspection_policy = NodesAndLinkedEdges()

plot.add_tools(HoverTool(tooltips="@index"))

# Add a title for the entire visualization using Div
plot_html = """<h3>Chart 1: Ticket-to-Ride Network Structure</h3>
Note: Nodes represent cities and edges represent tracks between cities, node size is proportional to the degree
<br>
<i>Hover over each node for more information</i>
"""
plot_subtitle = Div(width=800, text=plot_html)

# Visualize
show(column(plot_subtitle, plot))

#show(plot)
#save(plot)

### Degree & Weights Distribution

In [8]:
from collections import Counter 
degree_count = Counter(['Degree-' + str(G.degree[n]) for n in G.nodes(data=False)]) 
weights_count = Counter(['Weight-' + str(d['weight']) for u,v,d in G.edges(data=True)])

from math import pi
import pandas as pd
from bokeh.plotting import figure, show, save
from bokeh.io import output_file, output_notebook
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.layouts import row, column
from bokeh.models import Div

output_file(output_dir + "degree_weight_dist.html")
#output_notebook()

degrees = dict(degree_count)
p1_data = pd.Series(degrees).reset_index(name='value').rename(columns={'index':'degree'})
p1_data['angle'] = p1_data['value']/p1_data['value'].sum() * 2*pi
p1_data['pct_share'] = round((p1_data['value']/p1_data['value'].sum())*100,2)
p1_data['color'] = Category20c[len(degrees)]

p1 = figure(plot_height=350, plot_width=500, title="2a-Degree Distribution", toolbar_location=None,
           tools="hover", tooltips="@degree: @value (@pct_share%)", x_range=(-0.5, 1.0))

p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='degree', source=p1_data)

p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None
p1.legend.location = "bottom_right"

#show(p1)

weights = dict(weights_count)
p2_data = pd.Series(weights).reset_index(name='value').rename(columns={'index':'weight'})
p2_data['angle'] = p2_data['value']/p2_data['value'].sum() * 2*pi
p2_data['pct_share'] = round((p2_data['value']/p2_data['value'].sum())*100,2)
p2_data['color'] = Category20c[len(weights)]

p2 = figure(plot_height=350, plot_width=500, title="2b-Weight Distribution", toolbar_location=None,
           tools="hover", tooltips="@weight: @value (@pct_share%)", x_range=(-0.5, 1.0))

p2.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='weight', source=p2_data)

p2.axis.axis_label=None
p2.axis.visible=False
p2.grid.grid_line_color = None
p2.legend.location = "bottom_right"

# Add a title for the entire visualization using Div
html = """<h3>Chart 2: Degree and Weight Distribution</h3>
Note: Each pie segment represents the number of cities with a corresponding degree or weight.
<br>
<i>Hover over each pie segment for more information</i>
"""
subtitle = Div(width=800, text=html)
show(column(subtitle, row(p1, p2)))

#show(p2)
#show(row(p1, p2))


In [9]:
degree_df = pd.DataFrame([[n,v] for n,v in sorted(Gm.degree, key=lambda x: x[1], reverse=True)], columns=['city','degree'])

#display(HTML(degree_df.sort_values(['degree'],ascending=False).head(10).to_html(index=False)))
#display(HTML(degree_df.sort_values(['degree'],ascending=True).head(10).to_html(index=False)))

### Basic Stats

In [10]:
# fraction of nodes with degree < 5
len(degree_df[degree_df.degree<5])/len(degree_df)*100

72.3404255319149

In [11]:
# fraction of nodes with degree < 4
len(degree_df[degree_df.degree<4])/len(degree_df)*100

40.42553191489361

In [12]:
# fraction of nodes with degree < 3
len(degree_df[degree_df.degree<3])/len(degree_df)*100

10.638297872340425

In [13]:
nx.radius(G)

5

In [14]:
nx.diameter(G)

9

In [15]:
nx.center(G)

['Berlin', 'Venezia', 'Munchen']

In [16]:
nx.periphery(G)

['Lisboa', 'Cadiz', 'Edinburgh', 'Erzurum', 'Sochi', 'Rostov', 'Moskva']

In [17]:
ecc = nx.eccentricity(G)

# ecc = dict(sorted(ecc.items(), key=lambda x: x[1], reverse=True))
# print(json.dumps(ecc, indent=4))

temp = list()
for k,v in ecc.items():
    temp.append((k,v))
    
ecc_df = pd.DataFrame.from_records(temp, columns=['city','eccentricity'])
del temp

#display(HTML(ecc_df[ecc_df.eccentricity==nx.diameter(G)].sort_values(['eccentricity'],ascending=False).head(10).to_html(index=False)))
#display(HTML(ecc_df[ecc_df.eccentricity==nx.radius(G)].sort_values(['eccentricity'],ascending=True).head(10).to_html(index=False)))

In [18]:
# listing paths with length as diameter from all peripheries
diameter = nx.diameter(G)
for n in nx.periphery(G):
    paths = nx.shortest_path(G,source=n)
    for k,v in paths.items():
        if(len(v)-1==diameter):
            print("-".join(v))

Lisboa-Madrid-Barcelona-Marseille-Roma-Palermo-Smyrna-Angora-Erzurum-Sochi
Lisboa-Madrid-Barcelona-Marseille-Roma-Palermo-Smyrna-Constantinople-Sevastopol-Rostov
Lisboa-Madrid-Pamplona-Paris-Frankfurt-Essen-Kobenhavn-Stockholm-Petrograd-Moskva
Cadiz-Madrid-Barcelona-Marseille-Roma-Palermo-Smyrna-Angora-Erzurum-Sochi
Cadiz-Madrid-Barcelona-Marseille-Roma-Palermo-Smyrna-Constantinople-Sevastopol-Rostov
Cadiz-Madrid-Pamplona-Paris-Frankfurt-Essen-Kobenhavn-Stockholm-Petrograd-Moskva
Edinburgh-London-Dieppe-Paris-Marseille-Roma-Palermo-Smyrna-Angora-Erzurum
Edinburgh-London-Amsterdam-Essen-Berlin-Warszawa-Kyiv-Bucuresti-Sevastopol-Sochi
Erzurum-Angora-Smyrna-Palermo-Roma-Marseille-Paris-Dieppe-London-Edinburgh
Sochi-Erzurum-Angora-Smyrna-Palermo-Roma-Marseille-Barcelona-Madrid-Lisboa
Sochi-Erzurum-Angora-Smyrna-Palermo-Roma-Marseille-Barcelona-Madrid-Cadiz
Sochi-Sevastopol-Bucuresti-Budapest-Wien-Berlin-Essen-Amsterdam-London-Edinburgh
Rostov-Sevastopol-Constantinople-Smyrna-Palermo-Roma-M

In [19]:
# construct destination card points list
card_points_list=list()
with open(destinations_path) as f:
    for line in f:
        source, destination, card_points = pattern.findall(line)
        card_points=int(card_points)
        card_points_list.append(card_points)

## bar chart

from collections import Counter 
#card_points = [row['card_points'] for idx,row in destinations_df.iterrows()]
col_count = Counter(card_points_list) 

from bokeh.plotting import figure, show, save
from bokeh.io import output_file, output_notebook
from bokeh.models import HoverTool
from bokeh.layouts import row, column
from bokeh.models import Div

#output_notebook()
output_file(output_dir + "destination_card_points_dist.html")

labels = [str(key) for key,value in sorted(col_count.items())]
values = [value for key,value in sorted(col_count.items())]
pct_share = [round(value/len(card_points_list)*100,2) for key,value in sorted(col_count.items())]
source = ColumnDataSource(data = dict(labels = labels, values= values, pct_share=pct_share))
                          
p = figure(x_range=labels, plot_height=350, title=None)
p.vbar(x="labels", top="values", width=0.9, source=source)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.add_tools(HoverTool(tooltips=[("card points","@labels"),("count","@values (@pct_share%)")]))                          

#show(p)

# Add a title for the entire visualization using Div
html = """<h3>Chart 3: Destination Card Points Distribution</h3>
<i>Hover over each bar for more information</i>
"""
subtitle = Div(width=800, text=html)
show(column(subtitle, p))


### Importance of certain cities

In [20]:
# degree centrality
dc = nx.degree_centrality(G)
#top_dc = dict(sorted(dc.items(), key=lambda x: x[1], reverse=True)[:5])

temp = list()
for k,v in dc.items():
    temp.append((k,v))
    
dc_df = pd.DataFrame.from_records(temp, columns=['city','dc'])
del temp

#display(HTML(dc_df.sort_values(['dc'],ascending=False).head(10).to_html(index=False)))

In [21]:
# closeness centrality
cc = nx.closeness_centrality(G, distance='weight')

temp = list()
for k,v in cc.items():
    temp.append((k,v))
    
cc_df = pd.DataFrame.from_records(temp, columns=['city','cc'])
del temp

#display(HTML(cc_df.sort_values(['cc'],ascending=False).head(10).to_html(index=False)))

In [22]:
# betweenness centrality
bc = nx.betweenness_centrality(G, weight='weight')

temp = list()
for k,v in bc.items():
    temp.append((k,v))
    
bc_df = pd.DataFrame.from_records(temp, columns=['city','bc'])
del temp

#display(HTML(bc_df.sort_values(['bc'],ascending=False).head(10).to_html(index=False)))

In [23]:
# convert multigraph to simple graph            
G = nx.Graph(Gm)

# node clustering co-efficient for each city
# definition: http://wandora.org/wiki/Clustering_coefficient
ccoef = nx.clustering(G)

temp = list()
for k,v in ccoef.items():
    temp.append((k,v))
    
ccoef_df = pd.DataFrame.from_records(temp, columns=['city','clustering_coefficient'])
del temp

#display(HTML(ccoef_df.sort_values(['clustering_coefficient'],ascending=False).head(10).to_html(index=False)))
#display(HTML(ccoef_df.sort_values(['clustering_coefficient'],ascending=True).head(10).to_html(index=False)))

In [24]:
# Merge cities dataframes
cities_df = pd.merge(degree_df, ecc_df, on="city")
cities_df = pd.merge(cities_df, dc_df, on="city")
cities_df = pd.merge(cities_df, bc_df, on="city")
cities_df = pd.merge(cities_df, cc_df, on="city")
cities_df = pd.merge(cities_df, ccoef_df, on="city")

In [25]:
display(HTML(cities_df.to_html(index=False)))

city,degree,eccentricity,dc,bc,cc,clustering_coefficient
Paris,7,7,0.152174,0.137203,0.100656,0.285714
Kyiv,6,7,0.130435,0.127660,0.093496,0.200000
Frankfurt,6,6,0.130435,0.206094,0.113022,0.266667
Pamplona,5,7,0.108696,0.060386,0.081129,0.400000
Marseille,5,7,0.108696,0.118542,0.097665,0.300000
Sevastopol,5,8,0.108696,0.055753,0.078498,0.300000
Constantinople,5,8,0.108696,0.073816,0.087287,0.300000
Bucuresti,5,8,0.108696,0.154723,0.101545,0.300000
Wilno,5,7,0.108696,0.073269,0.091089,0.300000
Warszawa,5,6,0.108696,0.134599,0.101099,0.300000


In [26]:
# measuring each node importance by change in connectivity, efficiency

G_avg_sp_cost = nx.average_shortest_path_length(G, weight='weight')
G_avg_node_connectivity = nx.average_node_connectivity(G)
G_global_efficiency = nx.global_efficiency(G)
G_avg_clustering = nx.average_clustering(G)

temp=list()
for u,d in G.nodes(data=True):
    H = nx.Graph(G)
    H.remove_node(u)
    
    H_avg_node_connectivity = nx.average_node_connectivity(H)
    H_global_efficiency = nx.global_efficiency(H)
    H_avg_clustering = nx.average_clustering(H)
    
    is_connected = 1 if nx.is_connected(H) else 0
    if(is_connected == 0):
        Hc = max(nx.connected_component_subgraphs(H), key=len)
        H_avg_sp_cost = nx.average_shortest_path_length(Hc, weight='weight')
        unreachable_nodes = len(H)-len(Hc)
    else:
        H_avg_sp_cost = nx.average_shortest_path_length(H, weight='weight')
        unreachable_nodes = 0
    
    avg_sp_cost_change = round((H_avg_sp_cost-G_avg_sp_cost)/G_avg_sp_cost * 100, 2)
    connectivity_change = round((H_avg_node_connectivity-G_avg_node_connectivity)/G_avg_node_connectivity * 100, 2)
    global_efficiency_change = round((H_global_efficiency-G_global_efficiency)/G_global_efficiency*100, 2)
    clustering_coeff_change = round((H_avg_clustering-G_avg_clustering)/G_avg_clustering * 100, 2)
    temp.append((u,is_connected,unreachable_nodes,avg_sp_cost_change,connectivity_change,
                 clustering_coeff_change,global_efficiency_change))

node_importance_df = pd.DataFrame.from_records(temp, columns=['city','is_connected','unreachable_nodes','avg_sp_cost_change','avg_connectivity_change',
                                                              'clustering_coeff_change','global_efficiency_change'])
del temp

In [49]:
display(HTML(node_importance_df.to_html(index=False)))

city,is_connected,unreachable_nodes,avg_sp_cost_change,avg_connectivity_change,clustering_coeff_change,global_efficiency_change
Lisboa,1,0,-2.16,2.81,-11.84,1.43
Cadiz,1,0,-2.16,2.81,-11.84,1.43
Madrid,0,2,-6.15,-1.60,-11.14,-4.30
Barcelona,1,0,0.06,0.00,-2.73,0.41
Pamplona,1,0,0.88,-7.08,-4.03,-1.03
Marseille,1,0,2.77,-10.35,2.97,-2.20
Paris,1,0,2.14,-6.97,-9.17,-3.16
Brest,1,0,-0.28,-4.24,-1.93,0.24
Dieppe,1,0,0.69,-6.97,3.21,-0.32
London,0,1,-1.51,-1.70,6.84,-2.10


In [29]:
# from IPython.display import display_html 

# df1 = pd.DataFrame(np.arange(12).reshape((3,4)),columns=['A','B','C','D',])
# df2 = pd.DataFrame(np.arange(16).reshape((4,4)),columns=['A','B','C','D',])

# df1_styler = df1.style.set_table_attributes("style='display:inline;margin:10px;'").set_caption('Table 1')
# df2_styler = df2.style.set_table_attributes("style='display:inline;margin:10px;'").set_caption('Table 2')

# display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

### Importance of certain edge segments

In [47]:
# edge-betweenness centrality
ebc = nx.edge_betweenness_centrality(G,weight='weight')
#nx.set_edge_attributes(G, ebc, 'betweenness')

temp = list()
for k,v in ebc.items():
    temp.append((k,v))
    
ebc_df = pd.DataFrame.from_records(temp, columns=['segment','edge_betweenness'])
del temp

display(HTML(ebc_df.sort_values(['edge_betweenness'],ascending=False).head(10).to_html(index=False)))
display(HTML(ebc_df.sort_values(['edge_betweenness'],ascending=True).tail(10).to_html(index=False)))

segment,edge_betweenness
"(Budapest, Wien)",0.142879
"(Zagrab, Venezia)",0.131673
"(Wien, Munchen)",0.117440
"(Frankfurt, Munchen)",0.112625
"(Bucuresti, Budapest)",0.110365
"(Venezia, Zurich)",0.105832
"(Marseille, Zurich)",0.103321
"(Berlin, Frankfurt)",0.098806
"(Barcelona, Marseille)",0.097595
"(Roma, Venezia)",0.090287


segment,edge_betweenness
"(Roma, Venezia)",0.090287
"(Barcelona, Marseille)",0.097595
"(Berlin, Frankfurt)",0.098806
"(Marseille, Zurich)",0.103321
"(Venezia, Zurich)",0.105832
"(Bucuresti, Budapest)",0.110365
"(Frankfurt, Munchen)",0.112625
"(Wien, Munchen)",0.117440
"(Zagrab, Venezia)",0.131673
"(Budapest, Wien)",0.142879


In [31]:
# measuring each edge importance by change in connectivity, efficiency

temp=list()
for u,v,d in G.edges(data=True):
    H = nx.Graph(G)
    H.remove_edge(u,v)
    
    H_avg_node_connectivity = nx.average_node_connectivity(H)
    H_global_efficiency = nx.global_efficiency(H)
    H_avg_clustering = nx.average_clustering(H)
    
    is_connected = 1 if nx.is_connected(H) else 0
    if(is_connected == 0):
        Hc = max(nx.connected_component_subgraphs(H), key=len)
        H_avg_sp_cost = nx.average_shortest_path_length(Hc, weight='weight')
        unreachable_nodes = len(H)-len(Hc)
    else:
        H_avg_sp_cost = nx.average_shortest_path_length(H, weight='weight')
        unreachable_nodes = 0
    
    avg_sp_cost_change = round((H_avg_sp_cost-G_avg_sp_cost)/G_avg_sp_cost * 100, 2)
    connectivity_change = round((H_avg_node_connectivity-G_avg_node_connectivity)/G_avg_node_connectivity * 100, 2)
    global_efficiency_change = round((H_global_efficiency-G_global_efficiency)/G_global_efficiency*100, 2)
    clustering_coeff_change = round((H_avg_clustering-G_avg_clustering)/G_avg_clustering * 100, 2)
    temp.append(('{}-{}'.format(u,v),is_connected,unreachable_nodes,avg_sp_cost_change,connectivity_change,
                 clustering_coeff_change,global_efficiency_change))

    edge_importance_df = pd.DataFrame.from_records(temp, columns=['segment','is_connected','unreachable_nodes','avg_sp_cost_change','avg_connectivity_change',
                                                              'clustering_coeff_change','global_efficiency_change'])
del temp

In [50]:
display(HTML(edge_importance_df.to_html(index=False)))

segment,is_connected,unreachable_nodes,avg_sp_cost_change,avg_connectivity_change,clustering_coeff_change,global_efficiency_change
Lisboa-Cadiz,1,0,0.03,-0.10,-14.85,-0.14
Lisboa-Madrid,1,0,0.72,-0.10,-13.71,-0.57
Cadiz-Madrid,1,0,0.72,-0.10,-13.71,-0.57
Madrid-Barcelona,1,0,0.62,-1.41,1.60,-0.23
Madrid-Pamplona,1,0,0.36,-1.41,-1.60,-0.89
Barcelona-Pamplona,1,0,0.20,-0.03,-6.86,-0.18
Barcelona-Marseille,1,0,0.95,-2.69,1.83,-0.45
Pamplona-Marseille,1,0,0.22,-0.03,-3.98,-0.30
Pamplona-Paris,1,0,0.99,-1.21,-3.56,-0.86
Pamplona-Brest,1,0,0.12,-5.43,2.64,-0.27


### Route Profitability

In [34]:
# construct shortest paths of destinations
temp=list()
with open(destinations_path) as f:
    for line in f:
        source, destination, card_points = pattern.findall(line)
        card_points=int(card_points)
        p1=list()
        points1=0
        distance1=0
        length1=0
        for p in nx.all_shortest_paths(G,source,destination,weight='weight'):
            _points=points_of_path(G,p)
            if(_points>points1):
                points1=_points
                distance1=distance_of_path(G,p)
                p1=p
                length1=len(p)
        total_points=card_points+points1
        temp.append((source,destination,card_points,length1,distance1,total_points,'-'.join(p1)))
        
sp_df = pd.DataFrame.from_records(temp, columns=['source','destination','card_points',
                                                           'sp_length','sp_cost','sp_total_points','sp_path'])
del temp

#  calculate points-per-capita
sp_df['sp_profitability'] = sp_df.apply(lambda row: round(row['sp_total_points']/row['sp_cost'],2), axis=1)

# connectivity
sp_df['connectivity'] = sp_df.apply(lambda row: nx.edge_connectivity(G,row['source'],row['destination']), axis=1)

In [35]:
display(HTML(sp_df.sort_values(['sp_total_points'],ascending=False).to_html(index=False)))
# display(HTML(sp_df.sort_values(['connectivity'],ascending=False).to_html(index=False)))

source,destination,card_points,sp_length,sp_cost,sp_total_points,sp_path,sp_profitability,connectivity
Palermo,Moskva,20,7,20,54,Palermo-Smyrna-Constantinople-Bucuresti-Kyiv-Smolensk-Moskva,2.70,3
Kobenhavn,Erzurum,21,8,21,53,Kobenhavn-Essen-Berlin-Wien-Budapest-Bucuresti-Sevastopol-Erzurum,2.52,2
Cadiz,Stockholm,21,8,21,50,Cadiz-Madrid-Pamplona-Paris-Frankfurt-Essen-Kobenhavn-Stockholm,2.38,2
Brest,Petrograd,20,7,20,50,Brest-Paris-Frankfurt-Berlin-Warszawa-Wilno-Petrograd,2.50,3
Lisboa,Danzig,20,7,20,50,Lisboa-Madrid-Pamplona-Paris-Frankfurt-Berlin-Danzig,2.50,2
Edinburgh,Athina,21,9,20,48,Edinburgh-London-Dieppe-Paris-Zurich-Venezia-Roma-Brindisi-Athina,2.40,1
Frankfurt,Smolensk,13,5,13,32,Frankfurt-Berlin-Warszawa-Wilno-Smolensk,2.46,3
Amsterdam,Wilno,12,5,12,29,Amsterdam-Frankfurt-Berlin-Warszawa-Wilno,2.42,4
Berlin,Moskva,12,5,12,29,Berlin-Warszawa-Wilno-Smolensk-Moskva,2.42,3
Essen,Kyiv,10,4,10,26,Essen-Berlin-Warszawa-Kyiv,2.60,4


### Alternate Scoring Strategies

In [51]:
# compare alternate paths between given cities
temp = list()
u = 'Palermo'
v = 'Moskva'
cutoff = nx.shortest_path_length(G, u, v) + 2
paths = alternate_scoring_paths_with_cutoff(G, u, v, cutoff)
for p in paths:
    length1 = len(p)
    cost1 = distance_of_path(G, p)
    points1 = int(sp_df[(sp_df['source']==u) & (sp_df['destination']==v)]['card_points'])
    points2 = points_of_path(G, p)
    total_points = points1 + points2
    per_capita=round(total_points/cost1,2)
    temp.append((p,length1,cost1,total_points,per_capita))

temp_df = pd.DataFrame.from_records(temp,columns=['path','path_length','path_cost','path_total_points','path_points_per_capita'])
del temp

# temp_df.sort_values(['path_points_per_capita'],ascending=False).head(5)
display(HTML(temp_df.sort_values(['path_points_per_capita'],ascending=False).to_html(index=False)))

path,path_length,path_cost,path_total_points,path_points_per_capita
"[Palermo, Smyrna, Constantinople, Sevastopol, Rostov, Kharkov, Moskva]",7,22,60,2.73
"[Palermo, Smyrna, Constantinople, Bucuresti, Kyiv, Smolensk, Moskva]",7,20,54,2.70
"[Palermo, Smyrna, Constantinople, Bucuresti, Kyiv, Kharkov, Moskva]",7,23,62,2.70
"[Palermo, Smyrna, Constantinople, Bucuresti, Budapest, Kyiv, Kharkov, Moskva]",8,29,77,2.66
"[Palermo, Smyrna, Athina, Sarajevo, Budapest, Kyiv, Kharkov, Moskva]",8,29,77,2.66
"[Palermo, Smyrna, Athina, Sarajevo, Budapest, Kyiv, Smolensk, Moskva]",8,26,69,2.65
"[Palermo, Smyrna, Constantinople, Bucuresti, Budapest, Kyiv, Smolensk, Moskva]",8,26,69,2.65
"[Palermo, Smyrna, Constantinople, Sevastopol, Sochi, Rostov, Kharkov, Moskva]",8,22,57,2.59
"[Palermo, Roma, Venezia, Zagrab, Budapest, Kyiv, Kharkov, Moskva]",8,24,62,2.58
"[Palermo, Smyrna, Constantinople, Sevastopol, Bucuresti, Kyiv, Kharkov, Moskva]",8,28,72,2.57


In [52]:
# #histogram of all points for all alternate paths

# fig = plt.figure(figsize=(12,5))

# plt.subplot(1, 2, 1)
# plt.hist([temp_df['path_total_points']], label="Points")
# plt.legend(loc='upper right')
# plt.title('Palermo-Moskva Alternate paths \n Total Points distribution')

# plt.subplot(1, 2, 2)
# plt.hist([temp_df['path_cost']], label="Cost")
# plt.legend(loc='upper right')
# plt.title('Palermo-Moskva Alternate paths \n Cost for points distribution')

# plt.tight_layout()

### Route Blocking Strategies

In [57]:
for min_cut in list(nx.minimum_edge_cut(G,'Bucuresti','Berlin')):
    print(min_cut)                   

('Warszawa', 'Berlin')
('Danzig', 'Berlin')
('Frankfurt', 'Berlin')
('Essen', 'Berlin')
('Wien', 'Berlin')


In [59]:
for min_cut in list(nx.minimum_edge_cut(G,'Amsterdam','Wilno')):
    print(min_cut)  

('Amsterdam', 'Essen')
('London', 'Dieppe')
('Amsterdam', 'Frankfurt')
('Amsterdam', 'Bruxelles')


In [58]:
for min_cut in list(nx.minimum_edge_cut(G,'Edinburgh','Athina')):
    print(min_cut)  

('Edinburgh', 'London')
